In [1]:
# bibliotecas básicas
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

%matplotlib inline

In [2]:
# função para traçar reta no plano cartesiano (x2 = f(x1))
def coord(w0,w1,w2,x1):
  if w2 != 0:
    return (-w0-w1*x1)/w2
  else:
    return 2000 # Inf

In [3]:
# dataset Iris
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)

df = pd.get_dummies(df, columns=[4])

# as classes
y = df.iloc[0:150, [4,5,6]].values

# seleciona features (colunas) de entrada
X = df.iloc[0:150, [0,1,2,3]].values

df.head()

,0,1,2,3,4_Iris-setosa,4_Iris-versicolor,4_Iris-virginica
0,5.1,3.5,1.4,0.2,True,False,False
1,4.9,3.0,1.4,0.2,True,False,False
2,4.7,3.2,1.3,0.2,True,False,False
3,4.6,3.1,1.5,0.2,True,False,False
4,5.0,3.6,1.4,0.2,True,False,False


In [4]:
# base
# classe Neuronio e aprendizado estocastico/online

class Neuronio(object):

    def __init__(self, eta=0.1, epocas=50, fativ='perceptron',showErro=False):
        self.eta = eta
        self.epocas = epocas
        self.w_ = np.random.rand(1 + X.shape[1]) - 0.5
        self.fativ = fativ
        self.showErro = showErro
        self.erro_ep = 0
        self.SqError_ = []

    def somat(self, X):
        return self.w_[0] + np.dot(X, self.w_[1:])

    def sinal(self, x):
        return np.where(x >= 0.0, 1, -1)

    def tanh(self, x):
      return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def d_tanh(self, x):
      return 1 - self.tanh(x) * self.tanh(x)

    def ReLU(self, x):
      return np.where(x > 0.0, x, 0.1*x)  #max(0, x) .... float

    def d_ReLU(self, x):
      return np.where(x > 0.0, 1, 0.1)

    def sigmoide(self, x):
      return 1/(1 + np.exp(-x))

    def d_sigmoide(self, x):
      return self.sigmoide(x)*(1-self.sigmoide(x))

    def predict(self,X):
      if self.fativ == 'sinal' or self.fativ == 'perceptron':
#        print(f"sinalPredict={self.sinal(self.somat(X))}")  # debug
        return self.sinal(self.somat(X))
      elif self.fativ == 'linear':
        return self.somat(X)
      elif self.fativ == 'tanh':
        return self.tanh(self.somat(X))
      elif self.fativ == 'ReLU':
        return self.ReLU(self.somat(X))
      elif self.fativ == 'sigmoide':
        return self.sigmoide(self.somat(X))
      else:
        return 11

    def deltaW(self, erro):
      if self.fativ == 'sinal' or self.fativ == 'perceptron':
        atualiza = self.eta * erro
      elif self.fativ == 'linear':
        atualiza = self.eta * erro
      elif self.fativ == 'tanh':
#        print(f"> eta={self.eta} tanh={self.tanh(erro)} dtanh={self.d_tanh(erro)} erro={erro}")
        atualiza = self.eta * self.d_tanh(erro) * erro
      elif self.fativ == 'ReLU':
        atualiza = self.eta * self.d_ReLU(erro) * erro
      elif self.fativ == 'sigmoide':
        atualiza = self.eta * self.d_sigmoide(erro) * erro
      else:
        print(f"Função de ativação '{self.fativ}' desconhecida")
        exit()
      return atualiza

    def treinaGD(self,X,y):
      self.SqError_ = []
      self.erros_classif_ = []
      for ep in range(self.epocas):
        erro_classif = 0
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
        Xs = X[indices]
        ys = y[indices]
        outputs = self.predict(Xs)
        erros = (ys - outputs)
       # print(outputs,erros)
        erro_classif = np.where(ys*outputs < 0, 1, 0).sum()
        self.erros_classif_.append(erro_classif)
        if ep < (self.epocas-1):
          self.w_[1:] += self.eta * Xs.T.dot(erros)
          self.w_[0] += self.eta * erros.sum()
        SqError = (erros**2).sum() / 2.0 # ou np.square(erros)/2
        self.SqError_.append(SqError)
        if self.showErro:
          if ep == 0:
            print(f"{'Época':^10}\tErro")
          else:
            print(f"{str(ep):^10}\t{erro_classif}")
      return self

    def treinaSGD(self, X, y):
        self.erros_ = []
        self.erros_classif_ = []
        self.SqError_ = []
        for ep in range(self.epocas):
            erro_ep = 0
            erro_classif = 0
            SqError = 0

            indices = np.arange(X.shape[0])
            np.random.shuffle(indices)
            Xs = X[indices]
            ys = y[indices]

            for xi, target in zip(Xs, ys):
              output = self.predict(xi)
              erro = target - output
              SqError += erro*erro
              erro_ep += erro
              if (float(target)*float(output)) < 0:
                #print(f"ep={ep} {xi} target={target} output={output} t*o={target*output}") # debug
                erro_classif += 1

              if (ep < (self.epocas-1)):  #
                atualizacao = self.deltaW(erro)
                self.w_[0] +=  atualizacao * 1
                self.w_[1:] +=  atualizacao * xi

            self.erros_.append(erro_ep)
            self.SqError_.append(SqError)
            self.erros_classif_.append(erro_classif)
            if self.showErro:
              if ep == 0:
                print(f"{'Época':^10}\tErro")
              else:
                print(f"{str(ep):^10}\t{erro_classif}")

        return self

In [5]:
neur1 = Neuronio(fativ='sigmoide', eta=0.1, epocas=1000,showErro=False)
neur2 = Neuronio(fativ='sigmoide', eta=0.1, epocas=1000,showErro=False)
neur3 = Neuronio(fativ='sigmoide', eta=0.1, epocas=1000,showErro=False)
neur1.treinaSGD(X,y[:,0])
neur2.treinaSGD(X,y[:,1])
neur3.treinaSGD(X,y[:,2])

In [6]:
entrada = X[122]
neur1.predict(entrada),neur2.predict(entrada),neur3.predict(entrada)

(1.0375155524986666e-10, 0.6197727739057752, 0.9999348655451198)

In [7]:
def ProbClasse(x):
  p1 = neur1.predict(x)
  p2 = neur2.predict(x)
  p3 = neur3.predict(x)
  soma = p1+p2+p3
  return p1/soma, p2/soma, p3/soma

In [8]:
def Classif(x):
  probs = (neur1.predict(x), neur2.predict(x), neur3.predict(x))
  idx = np.array(probs).argmax()
  classes = ['4_Iris-setosa',	'Iris-versicolor',	'Iris-virginica']
  return classes[idx]

In [9]:
ProbClasse(entrada),Classif(entrada)

((6.405573000717221e-11, 0.3826448420507465, 0.6173551578851978),
 'Iris-virginica')